# 2024 데이터 크리에이터 캠프

문제: 인공지능은 사람의 마음을 이해할수 있을까?

## Mission3. 패션스타일 선호 여부 예측

## 라이브러리 불러오기

In [1]:
import os
import torch
import json
import numpy as np
import pandas as pd
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score
from typing import Type
from collections import defaultdict
import torchvision.transforms as transforms
from sklearn.metrics.pairwise import cosine_similarity

## Resnet

In [2]:
class BasicBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        stride: int = 1,
        expansion: int = 1,
        downsample: nn.Module = None
    ) -> None:
        super(BasicBlock, self).__init__()
        self.expansion = expansion
        self.downsample = downsample
        self.conv1 = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            out_channels,
            out_channels*self.expansion,
            kernel_size=3,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels*self.expansion)

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return  out

In [3]:
class ResNet(nn.Module):
    def __init__(
        self,
        img_channels: int,
        num_layers: int,
        block: Type[BasicBlock],
        num_classes: int  = 1000
    ) -> None:
        super(ResNet, self).__init__()
        if num_layers == 18: # ResNet18 만을 본 대회에서 사용함으로 18층만 구현
            layers = [2, 2, 2, 2]
            self.expansion = 1

        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            in_channels=img_channels,
            out_channels=self.in_channels,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*self.expansion, num_classes)

    def _make_layer(
        self,
        block: Type[BasicBlock],
        out_channels: int,
        blocks: int,
        stride: int = 1
    ) -> nn.Sequential:
        downsample = None
        if stride != 1:
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    out_channels*self.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(out_channels * self.expansion),
            )
        layers = []
        layers.append(
            block(
                self.in_channels, out_channels, stride, self.expansion, downsample
            )
        )
        self.in_channels = out_channels * self.expansion

        for i in range(1, blocks):
            layers.append(block(
                self.in_channels,
                out_channels,
                expansion=self.expansion
            ))
        return nn.Sequential(*layers)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        print('Dimensions of the last convolutional feature map: ', x.shape)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

### ResNet-18 모델 정의

주어진 ResNet-18 모델을 사용하여 각 이미지의 feature vector를 추출  
-> 프리트레인 가중치 사용함

In [4]:
# ResNet-18 모델 정의
class ResNet18FeatureExtractor(nn.Module):
    def __init__(self):
        super(ResNet18FeatureExtractor, self).__init__()
        self.resnet18 = models.resnet18(pretrained=True) # 사전 학습된 가중치
        self.features = nn.Sequential(*list(self.resnet18.children())[:-1])  # 마지막 FC 레이어 제외

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        return x

In [5]:
# CSV 파일 불러오기
mission2_result = pd.read_csv('../dataset/mission2-2_result_all.csv')

# 데이터프레임의 일부 출력
mission2_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3480 entries, 0 to 3479
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   응답자 ID     3480 non-null   int64 
 1   train 선호   2215 non-null   object
 2   train 비선호  2830 non-null   object
 3   valid 선호   1048 non-null   object
 4   valid 비선호  1354 non-null   object
dtypes: int64(1), object(4)
memory usage: 136.1+ KB


In [6]:
mission2_result.head()

,응답자 ID,train 선호,train 비선호,valid 선호,valid 비선호
0,58049,NaN,T_00253_60_popart_W.jpg,NaN,T_00253_60_popart_W.jpg
1,62192,NaN,T_00253_60_popart_W.jpg,NaN,T_00253_60_popart_W.jpg
2,64213,NaN,T_00253_60_popart_W.jpg,NaN,T_00253_60_popart_W.jpg
3,66592,"T_00253_60_popart_W.jpg, T_00893_90_hiphop_W.j...","T_07452_50_classic_W.jpg, W_02170_50_feminine_...","T_00253_60_popart_W.jpg, W_10028_50_classic_W....","W_02170_50_feminine_W.jpg, W_19352_50_feminine..."
4,66721,W_05960_70_hippie_W.jpg,T_00253_60_popart_W.jpg,NaN,T_00253_60_popart_W.jpg


- train 데이터에서 선호/비선호 여부에 따라 데이터 있는경우에 추천이 가능하다고 판단하였고  
- valid 데이터의 또한 선호/비선호 데이터가 모두 있는 경우를 사용  
-> valid 데이터의 경우 선호/비선호 값에 속하는 이미지가 모두 존재해야되지는 않지만 통일성을 위해  
데이터가 모두 존재하는 상황에서 아이템 기반 필터링(Item-Based Filtering) 방식 적용

In [7]:
# 모든 열의 값이 채워져 있는 행만 선택
filtered_mission2_result = mission2_result.dropna()

# 인덱스 재설정
filtered_mission2_result = filtered_mission2_result.reset_index(drop=True)

# 결과 출력
filtered_mission2_result

,응답자 ID,train 선호,train 비선호,valid 선호,valid 비선호
0,66592,"T_00253_60_popart_W.jpg, T_00893_90_hiphop_W.j...","T_07452_50_classic_W.jpg, W_02170_50_feminine_...","T_00253_60_popart_W.jpg, W_10028_50_classic_W....","W_02170_50_feminine_W.jpg, W_19352_50_feminine..."
1,66469,"T_00456_10_sportivecasual_M.jpg, T_00588_10_sp...","T_02958_19_normcore_M.jpg, T_06076_60_mods_M.j...","T_00456_10_sportivecasual_M.jpg, T_01123_90_hi...","W_24553_70_hippie_M.jpg, W_24647_70_hippie_M.j..."
2,66513,"T_05088_19_normcore_W.jpg, T_07416_19_lounge_W...","T_08306_10_sportivecasual_W.jpg, T_08918_19_no...",W_14828_50_classic_W.jpg,"T_06910_50_classic_W.jpg, W_10984_50_feminine_..."
3,58251,"T_08242_10_sportivecasual_W.jpg, T_08663_19_no...","T_11058_90_lingerie_W.jpg, T_12089_80_powersui...","T_14538_00_cityglam_W.jpg, W_00716_60_minimal_...","W_04101_19_lounge_W.jpg, W_08886_10_athleisure..."
4,48094,T_14538_00_cityglam_W.jpg,W_09479_70_hippie_W.jpg,T_14538_00_cityglam_W.jpg,W_09479_70_hippie_W.jpg
...,...,...,...,...,...
463,64621,W_29596_10_sportivecasual_M.jpg,W_24250_90_hiphop_M.jpg,W_28925_90_hiphop_M.jpg,W_24250_90_hiphop_M.jpg
464,66842,W_27765_60_mods_M.jpg,W_24352_70_hippie_M.jpg,W_27765_60_mods_M.jpg,W_16485_00_metrosexual_M.jpg
465,64772,"W_32595_60_mods_M.jpg, W_48687_60_mods_M.jpg","W_24470_70_hippie_M.jpg, W_24590_60_mods_M.jpg...","W_33329_50_ivy_M.jpg, W_48687_60_mods_M.jpg",W_24590_60_mods_M.jpg
466,64828,W_35091_80_powersuit_W.jpg,W_34027_10_sportivecasual_W.jpg,"W_22783_70_hippie_W.jpg, W_35091_80_powersuit_...",W_34027_10_sportivecasual_W.jpg


### 이미지 전처리

ResNet-18 모델을 사용하여 이미지의 feature vector를 추출 및 저장

In [8]:
# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 이미지 feature vector 추출 함수
def extract_features(image_path, model, transform):
    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        return None
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = model(image)
    return features.numpy().flatten()

# 이미지 디렉토리 경로
train_image_directory = '../dataset/training_image'
valid_image_directory = '../dataset/validation_image'

# ResNet-18 모델 초기화
model = ResNet18FeatureExtractor()
model.eval()

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet18FeatureExtractor(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affi

이미지 간 유사도를 계산 및 Validation 데이터 내 응답자의 스타일 선호 여부를 예측

In [9]:
import os
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 학습 이미지와 검증 이미지의 디렉토리 경로 설정
train_image_directory = '../dataset/training_image'
valid_image_directory = '../dataset/validation_image'

# 유사도 계산 함수
def calculate_similarity(feature1, feature2):
    return cosine_similarity([feature1], [feature2])[0][0]

# 응답자별 학습 이미지와 검증 이미지의 feature vector 추출 및 저장
def extract_respondent_features(mission2_result, model, transform):
    respondent_train_features = {}
    respondent_valid_features = {}

    for index, row in mission2_result.iterrows():
        respondent_id = row['응답자 ID']
        train_images = []
        valid_images = []

        if not pd.isna(row['train 선호']):
            train_images.extend([(img, '선호') for img in row['train 선호'].split(', ')])
        if not pd.isna(row['train 비선호']):
            train_images.extend([(img, '비선호') for img in row['train 비선호'].split(', ')])
        if not pd.isna(row['valid 선호']):
            valid_images.extend(row['valid 선호'].split(', '))
        if not pd.isna(row['valid 비선호']):
            valid_images.extend(row['valid 비선호'].split(', '))

        respondent_train_features[respondent_id] = {}
        respondent_valid_features[respondent_id] = {}

        for train_image, preference in train_images:
            image_path = os.path.join(train_image_directory, train_image).replace('\\', '/')
            if not os.path.exists(image_path):
                print(f"File not found: {image_path}")
                continue
            image_id = train_image.split('_')[1]
            feature = extract_features(image_path, model, transform)
            if feature is not None:
                respondent_train_features[respondent_id][image_id] = (feature, preference)

        for valid_image in valid_images:
            image_path = os.path.join(valid_image_directory, valid_image).replace('\\', '/')
            if not os.path.exists(image_path):
                print(f"File not found: {image_path}")
                continue
            image_id = valid_image.split('_')[1]
            feature = extract_features(image_path, model, transform)
            if feature is not None:
                respondent_valid_features[respondent_id][image_id] = feature

    return respondent_train_features, respondent_valid_features

# 응답자별 학습 이미지와 검증 이미지의 feature vector 추출
respondent_train_features, respondent_valid_features = extract_respondent_features(filtered_mission2_result, model, transform)

# Validation 데이터 내 응답자의 스타일 선호 여부 예측
def predict_preference(respondent_train_features, respondent_valid_features, threshold=0.5):
    predictions = {}
    for respondent_id, valid_features in respondent_valid_features.items():
        respondent_predictions = {}
        for valid_image_id, valid_feature in valid_features.items():
            similarities = {'선호': [], '비선호': []}
            for train_image_id, (train_feature, preference) in respondent_train_features[respondent_id].items():
                similarity = calculate_similarity(valid_feature, train_feature)
                similarities[preference].append(similarity)
            # 모든 유사도를 사용하여 평균 계산
            if len(similarities['선호']) == 0 and len(similarities['비선호']) == 0:
                preference_score = 0  # 기본값 설정
            else:
                preference_score = (sum(similarities['선호']) - sum(similarities['비선호'])) / (len(similarities['선호']) + len(similarities['비선호']))
            respondent_predictions[valid_image_id] = '선호' if preference_score > threshold else '비선호'
        predictions[respondent_id] = respondent_predictions
    return predictions

# 예측 수행
predictions = predict_preference(respondent_train_features, respondent_valid_features, threshold=0.5)

In [19]:
predictions = predict_preference(respondent_train_features, respondent_valid_features, threshold=0.5)

Accuracy: 0.56

성능 확인

In [20]:
# 성능 측정 (예시로 정확도 계산)
def calculate_accuracy(predictions, mission2_result):
    correct = 0
    total = 0
    for index, row in mission2_result.iterrows():
        respondent_id = row['응답자 ID']
        if respondent_id not in predictions:
            continue
        for valid_image in row['valid 선호'].split(', ') if not pd.isna(row['valid 선호']) else []:
            valid_image_id = valid_image.split('_')[1]
            if valid_image_id in predictions[respondent_id]:
                total += 1
                if predictions[respondent_id][valid_image_id] == '선호':
                    correct += 1
        for valid_image in row['valid 비선호'].split(', ') if not pd.isna(row['valid 비선호']) else []:
            valid_image_id = valid_image.split('_')[1]
            if valid_image_id in predictions[respondent_id]:
                total += 1
                if predictions[respondent_id][valid_image_id] == '비선호':
                    correct += 1
    return correct / total if total > 0 else 0

# 정확도 계산
accuracy = calculate_accuracy(predictions, mission2_result)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.56


.